In [9]:
from cobra import io,flux_analysis
from cobra.core import Reaction, Metabolite

#import model. Update file name and location in the next line
cobra_model = io.sbml.read_sbml_model("Data/PlantCoreMetabolism_v2_0_0.xml")
cobra_model.solver='glpk'

#Remove all metabolites except sucrose from Phloem
rxn = cobra_model.reactions.get_by_id("Phloem_output_tx")

#no external sucrose or glucose
cobra_model.reactions.get_by_id("Sucrose_tx").lower_bound = 0
cobra_model.reactions.get_by_id("Sucrose_tx").upper_bound = 0
cobra_model.reactions.get_by_id("GLC_tx").lower_bound = 0
cobra_model.reactions.get_by_id("GLC_tx").upper_bound = 0

#no external light energy
cobra_model.reactions.get_by_id("Photon_tx").lower_bound = 0
cobra_model.reactions.get_by_id("Photon_tx").upper_bound = 0

#set export of sugars as objective
cobra_model.reactions.get_by_id("Phloem_output_tx").objective_coefficient=1

#add source reaction for TP
rxn = Reaction("GAP_tx",name = "TP source")
rxn.add_metabolites({cobra_model.metabolites.get_by_id("GAP_c"):1})
rxn.upper_bound = 1000
rxn.lower_bound = 0
cobra_model.add_reaction(rxn)

#add source reaction for TP
rxn = Reaction("GLYCOLATE_tx",name = "Glycolate source")
rxn.add_metabolites({cobra_model.metabolites.get_by_id("GLYCOLLATE_c"):1})
rxn.upper_bound = 1000
rxn.lower_bound = 0
cobra_model.add_reaction(rxn)

#add source reaction for TP
rxn = Reaction("GLYCERATE_tx",name = "Glycerate sink")
rxn.add_metabolites({cobra_model.metabolites.get_by_id("GLYCERATE_c"):-1})
rxn.upper_bound = 1000
rxn.lower_bound = 0
cobra_model.add_reaction(rxn)

#remove mGS and cGS
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_m").lower_bound = 0
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_m").upper_bound = 0
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_c").lower_bound = 0
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_c").upper_bound = 0

#turn off phosphoserine transaminase
#cobra_model.reactions.get_by_id("PSERTRANSAM_RXN_p").lower_bound = 0
#cobra_model.reactions.get_by_id("PSERTRANSAM_RXN_p").upper_bound = 0

cobra_model.reactions.get_by_id("Pi_ec").lower_bound = -1000
cobra_model.reactions.get_by_id("Pi_ec").upper_bound = 1000

#add malate and citrate accumulation reactions
rxn = Reaction("MAL_v_accumulation")
rxn.add_metabolites({cobra_model.metabolites.MAL_v:-0.7,cobra_model.metabolites.aMAL_v:-0.3})
rxn.lower_bound = -1000
rxn.upper_bound = 1000
cobra_model.add_reaction(rxn)
rxn = Reaction("CIT_v_accumulation")
rxn.add_metabolites({cobra_model.metabolites.CIT_v:-0.5,cobra_model.metabolites.aCIT_v:-0.5})
rxn.lower_bound = -1000
rxn.upper_bound = 1000
cobra_model.add_reaction(rxn)


sol = flux_analysis.parsimonious.pfba(temp)



In [10]:
sol

,fluxes,reduced_costs
PRO_PROTON_vc,0.000000,-2.000000
Ca_tx,0.000000,-2.000000
H2O_xc,8.140373,-2.000000
sCIT_biomass,0.000000,2.000000
ACETYLGLUTKIN_RXN_p,0.000057,-2.000000
...,...,...
GAP_tx,1.587800,-747.846154
GLYCOLATE_tx,8.140600,-164.974359
GLYCERATE_tx,4.070300,591.897436
MAL_v_accumulation,0.177920,712.958974


In [14]:

temp = cobra_model.copy()

PPFD = 800
#constrain maintenace
ATPase = (0.0049*PPFD) + 2.7851
temp.reactions.get_by_id("ATPase_tx").lower_bound = ATPase
temp.reactions.get_by_id("ATPase_tx").upper_bound = ATPase

#constraint TP flux
temp.reactions.get_by_id("GAP_tx").lower_bound = 1.5878
temp.reactions.get_by_id("GAP_tx").upper_bound = 1.5878

#constraint glycollate and glycerate fluxes flux
temp.reactions.get_by_id("GLYCOLATE_tx").lower_bound = 8.1406
temp.reactions.get_by_id("GLYCOLATE_tx").upper_bound = 8.1406
temp.reactions.get_by_id("GLYCERATE_tx").lower_bound = 4.0703
temp.reactions.get_by_id("GLYCERATE_tx").upper_bound = 4.0703

#temp.reactions.get_by_id("NrefixationCostbypass").lower_bound = df270["Vt_glycolate"][i]
#temp.reactions.get_by_id("NrefixationCostbypass").upper_bound = df270["Vt_glycolate"][i]
#temp.reactions.get_by_id("NrefixationEnergy").lower_bound = df270["Vt_glycerate"][i]
#temp.reactions.get_by_id("NrefixationEnergy").upper_bound = df270["Vt_glycerate"][i]

temp.reactions.get_by_id("MAL_v_accumulation").lower_bound = 0.0698903487288*2.5457
temp.reactions.get_by_id("MAL_v_accumulation").upper_bound = 0.0698903487288*2.5457

temp.reactions.get_by_id("CIT_v_accumulation").lower_bound = -0.056884259879*2.5457
temp.reactions.get_by_id("CIT_v_accumulation").upper_bound = -0.056884259879*2.5457


#check if model works
sol = flux_analysis.parsimonious.optimize_minimal_flux(temp)
rxn =  temp.reactions.get_by_id("Phloem_output_tx")
met = temp.metabolites.sSUCROSE_b
print("Sucrose export rate ="+str(rxn.metabolites[met]*rxn.flux*-1))


Sucrose export rate =0.09918858007715127


In [30]:
total = 0
for rxn in temp.metabolites.ATP_p.reactions:
    if round(rxn.flux,3) != 0:
        coeff1 = rxn.metabolites[temp.metabolites.ATP_p]
        coeff2 = rxn.metabolites[temp.metabolites.aATP_p]
        ATPflux = rxn.flux*(coeff1+coeff2)
        #print(rxn.id+"\t"+str(ATPflux))
        if rxn.flux*(coeff1+coeff2)<0:
            total = total+abs(ATPflux)
print("Extra APTase flux ="+str(total))

Extra APTase flux =4.088747543949858
